In [13]:
from python_scripts.utils.classification_evals import *
from sklearn.linear_model import LogisticRegression, Perceptron, PassiveAggressiveClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
import warnings
# warnings.filterwarnings("ignore")

# Classification

In [14]:
final_separate_dataset_path = '../../../dataset/final/windows/separate/'
final_combined_dataset_path = '../../../dataset/final/windows/combined/'
acc_dataset_path = final_separate_dataset_path + 'PatchTable_Acceleration_Filtered.csv'
gyro_dataset_path = final_separate_dataset_path + 'PatchTable_Gyroscope_Filtered.csv'
acc_gyro_dataset_path = final_combined_dataset_path + 'PatchTable_Acceleration_Gyroscope_Filtered.csv'

In [15]:
acc_dataset_df = pd.read_csv(acc_dataset_path)
gyro_dataset_df = pd.read_csv(gyro_dataset_path)
acc_gyro_dataset_df = pd.read_csv(acc_gyro_dataset_path)
acc_id_groupings = acc_dataset_df['Subject']
gyro_id_groupings = gyro_dataset_df['Subject']
acc_gyro_id_groupings = acc_gyro_dataset_df['Subject']

In [16]:
acc_dataset_df

,Subject,Class,CorssCorrelationSum_VTAP,CorssCorrelationPeak_VTAP,IndexHarmonicity_ML,FirstDerivativeMean_SigComplete_VT,FirstDerivative_Moment3rd_SigComplete_VT,FirstDerivativeLogRatio_SigComplete_ML,FirstDerivativeMean_SigComplete_AP,FirstDerivative_Skweness_SigComplete_AP,...,ZeroCrossingRate_AP,Mean_SigComplete_Magnitude,Skewness_SigComplete_Magnitude,Percentile25_SigComplete_Magnitude,MaxFreqSpectrum_Below5Hz_Magnitude,RelativePower_BelowDomFr_Magnitude,MedianPSD_Plom_Magnitude,Moment3rdPSD_Plom_Magnitude,SkewnessPSD_Plom_Magnitude,ZeroCrossingRate_Magnitude
0,1,0,-0.163081,-0.662803,0.528986,0.475757,-0.062359,-0.136782,-0.409614,-1.334350,...,0.733053,-0.035799,1.275514,1.444325,-0.116140,-0.156583,-0.165272,-0.146060,-0.086591,-1.771846
1,1,0,-0.163483,-0.668085,-0.364654,0.090667,-0.110018,-0.233595,-0.054282,-1.066276,...,0.629589,-0.281341,-0.695417,1.432455,0.408305,-0.063859,-0.224007,-0.146061,0.952947,-2.360037
2,1,0,-0.163548,-0.668120,-0.362345,-0.033453,-0.110054,-0.112613,-0.050146,-1.160794,...,0.422661,-0.404884,-0.490985,1.412446,-2.039106,0.167928,-0.224137,-0.146061,1.178809,-2.654132
3,1,0,-0.163525,-0.668035,-0.376434,0.009309,-0.110064,-0.227149,-0.042921,-1.174912,...,-0.301586,-0.463610,-0.346706,1.393891,-2.039106,0.285549,-0.224224,-0.146061,1.203253,-2.801180
4,1,0,-0.163469,-0.667327,13.995720,-0.002560,-0.109835,-0.698540,0.162228,-0.055198,...,-0.508514,-0.208414,2.387815,1.407141,-1.951699,-0.016023,-0.196872,-0.146061,0.069855,-3.095275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1875,6,1,-0.152225,-0.607692,-0.416798,-0.867979,-0.047874,0.015603,-0.046169,0.002909,...,0.733053,0.185887,0.225318,0.462994,-0.465771,-0.191222,-0.209324,-0.146017,-0.295417,0.580916
1876,6,1,-0.150379,-0.623584,-0.154285,-0.524313,-0.062139,-0.028567,0.040108,0.172652,...,-0.301586,0.169964,0.472963,0.627831,-0.553178,-0.178239,-0.219464,-0.146033,0.136042,-0.007274
1877,6,1,-0.148807,-0.580757,-0.314912,0.828930,-0.047780,-0.325827,-0.340791,0.078664,...,-0.508514,0.280245,0.255260,0.389285,1.194972,-0.184609,-0.166815,-0.145960,-0.002798,0.139773
1878,6,1,-0.132307,-0.612842,-0.367696,1.526718,-0.039679,-0.599088,-2.453837,-3.100024,...,-1.025834,1.338794,0.849419,0.387508,1.194972,-0.083886,0.621261,-0.145660,-0.065654,-0.007274


In [17]:
gyro_dataset_df

,Subject,Class,RelativePower_Below5Hz_Magnitude
0,1,0,-2.247382
1,1,0,-2.401791
2,1,0,-0.511308
3,1,0,-0.713828
4,1,0,0.242117
...,...,...,...
2054,6,1,0.432625
2055,6,1,0.203724
2056,6,1,1.628756
2057,6,1,1.868931


In [18]:
acc_gyro_dataset_df

,Subject,Class,Acc_CorssCorrelationSum_VTAP,Acc_CorssCorrelationPeak_VTAP,Acc_FirstDerivativeRatio_SigComplete_AP,Acc_FirstDerivativeMax_SigComplete_Magnitude,Acc_InterQuartileRange_SigComplete_AP,Acc_RelativePower_BelowDomFr_Magnitude,Acc_ZeroCrossingRate_Magnitude,Gyro_FirstDerivativeLogRatio_SigComplete_ML,Gyro_FirstDerivativeLogRatio_SigComplete_AP,Gyro_RelativePower_Below5Hz_Magnitude
0,1,0,-0.166063,-0.685750,-0.010657,-1.074034,-1.444151,0.032532,-2.520228,-1.835525,-1.157528,-2.318428
1,1,0,-0.166134,-0.685786,0.041014,-1.074034,-1.475770,0.710991,-2.838854,-1.416182,-0.633449,-2.477687
2,1,0,-0.134558,-0.610375,-0.038005,-0.383370,0.146241,1.432397,-0.927096,-0.205820,0.317132,-0.527828
3,1,0,-0.129496,-0.603319,-0.040401,-0.383370,-0.003206,-0.400597,-0.767783,-0.149416,0.074079,-0.736708
4,1,0,-0.131921,-0.573847,0.089224,-0.383370,-0.061001,-0.376284,-0.927096,-0.441999,0.070623,0.214021
...,...,...,...,...,...,...,...,...,...,...,...,...
1733,6,1,-0.152122,-0.658918,-0.013876,-0.420395,-1.135367,-0.253517,0.825349,-0.285219,0.754709,0.274395
1734,6,1,-0.153823,-0.624051,-0.008705,-0.617524,-1.067617,-0.340272,0.666035,-0.315874,0.456793,0.445751
1735,6,1,-0.151816,-0.640286,-0.020001,-0.617524,-0.803723,-0.302268,0.028783,-0.030334,1.163076,1.679451
1736,6,1,-0.150106,-0.596533,-0.020139,-0.617524,-0.759067,-0.320913,0.188096,0.432560,1.242770,1.927169


In [19]:
acc_target = acc_dataset_df.iloc[:, 1].copy().values
gyro_target = gyro_dataset_df.iloc[:, 1].copy().values
acc_gyro_target = acc_gyro_dataset_df.iloc[:, 1].copy().values
acc_features = acc_dataset_df.iloc[:, 2:].copy().values
gyro_features = gyro_dataset_df.iloc[:, 2:].copy().values
acc_gyro_features = acc_gyro_dataset_df.iloc[:, 2:].copy().values

In [20]:
def get_model_scores(models_dict, features, target, id_groupings, groupk_folds):
    model_score_columns = list(scoring_methods.keys())
    model_score_columns.insert(0, 'Model')
    all_model_scores_df = pd.DataFrame(columns=model_score_columns)
    for model_key in models_dict:
        model_scores = cross_val_model(model=models_dict[model_key], features=features, target=target, id_groupings=id_groupings, groupk_folds=groupk_folds)
        model_scores['Model'] = model_key
        all_model_scores_df = pd.concat([all_model_scores_df, pd.DataFrame.from_dict(model_scores, orient='index').T], ignore_index=True)
    return all_model_scores_df

## Testing All Models

In [21]:
models = {
    'Logistic Regression': LogisticRegression(),
    'Support Vector Machine': SVC(kernel='sigmoid'),
    'Gaussian Naive Bayes': GaussianNB(),
    'Bernoulli Naive Bayes': BernoulliNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'Voting': VotingClassifier(estimators=[('lr', LogisticRegression()), ('dt', DecisionTreeClassifier()), ('knn', KNeighborsClassifier())]),
    'k-Nearest Neighbors': KNeighborsClassifier(n_neighbors=11),
    'Multi-layer Perceptron': MLPClassifier(max_iter=1000, hidden_layer_sizes=(2, 3, 2)),
    'Gaussian Process': GaussianProcessClassifier(),
    'Quadratic Discriminant Analysis': QuadraticDiscriminantAnalysis(),
    'Linear Discriminant Analysis': LinearDiscriminantAnalysis(),
    'Perceptron': Perceptron(),
    'Passive Aggressive': PassiveAggressiveClassifier()
}

In [22]:
all_model_scores_acc = get_model_scores(models, features=acc_features, target=acc_target, id_groupings=acc_id_groupings, groupk_folds=6)

/Library/Python/3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [23]:
all_model_scores_gyro = get_model_scores(models, features=gyro_features, target=gyro_target, id_groupings=gyro_id_groupings, groupk_folds=6)

/Library/Python/3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Python/3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Python/3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Python/3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defin

In [24]:
all_model_scores_acc_gyro = get_model_scores(models, features=acc_gyro_features, target=acc_gyro_target, id_groupings=acc_gyro_id_groupings, groupk_folds=6)

/Library/Python/3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
all_model_scores_acc

,Model,Accuracy,Precision,Sensitivity,Specificity,F1
0,Logistic Regression,0.630166,0.621803,0.62252,0.52811,0.614377
1,Support Vector Machine,0.642819,0.629674,0.615944,0.465882,0.602603
2,Gaussian Naive Bayes,0.573757,0.606191,0.580386,0.769069,0.538593
3,Bernoulli Naive Bayes,0.602113,0.600458,0.600051,0.496895,0.583766
4,Decision Tree,0.562367,0.529375,0.545769,0.473112,0.523891
5,Random Forest,0.564749,0.574689,0.543301,0.358616,0.526198
6,Gradient Boosting,0.551559,0.556623,0.546444,0.412452,0.527161
7,AdaBoost,0.548488,0.555938,0.556069,0.490396,0.537699
8,Extra Trees,0.592291,0.609124,0.566692,0.430112,0.566037
9,Voting,0.606043,0.598151,0.597097,0.480659,0.576775


In [26]:
all_model_scores_gyro

,Model,Accuracy,Precision,Sensitivity,Specificity,F1
0,Logistic Regression,0.600973,0.596594,0.556517,0.243546,0.516311
1,Support Vector Machine,0.491224,0.480892,0.480854,0.383172,0.46968
2,Gaussian Naive Bayes,0.60224,0.598914,0.556426,0.240303,0.517205
3,Bernoulli Naive Bayes,0.557671,0.333354,0.52052,0.130237,0.388592
4,Decision Tree,0.538741,0.523832,0.524615,0.431121,0.517526
5,Random Forest,0.538741,0.523832,0.524893,0.431121,0.517526
6,Gradient Boosting,0.585256,0.57093,0.538107,0.21267,0.492399
7,AdaBoost,0.57861,0.579416,0.540821,0.229367,0.487922
8,Extra Trees,0.536545,0.519953,0.517906,0.423779,0.515589
9,Voting,0.570168,0.545178,0.539206,0.320312,0.519353


In [27]:
all_model_scores_acc_gyro

,Model,Accuracy,Precision,Sensitivity,Specificity,F1
0,Logistic Regression,0.68087,0.66442,0.664469,0.591715,0.66226
1,Support Vector Machine,0.638351,0.636103,0.628499,0.578315,0.613428
2,Gaussian Naive Bayes,0.541568,0.59307,0.569729,0.836266,0.50946
3,Bernoulli Naive Bayes,0.649436,0.634087,0.635559,0.530298,0.625079
4,Decision Tree,0.588582,0.600824,0.583373,0.564195,0.591304
5,Random Forest,0.653977,0.629617,0.620114,0.544904,0.624179
6,Gradient Boosting,0.626677,0.614832,0.620711,0.562298,0.611727
7,AdaBoost,0.650552,0.639597,0.644885,0.636072,0.639263
8,Extra Trees,0.633742,0.620139,0.625722,0.54001,0.618861
9,Voting,0.64657,0.62392,0.629707,0.594703,0.630923
